In [5]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import math
from queue import PriorityQueue
import random

# List of all cities in the graph
cities = ['Panaji', 'Raichur', 'Tirupathi', 'Chennai', 'Mangalore', 
          'Kozhikode', 'Bangalore', 'Bellari', 'Kurnool', 'Nellore']

# Distances between cities as provided in the graph
distances = {
    ('Panaji', 'Mangalore'): 365,
    ('Mangalore', 'Panaji'): 365,
    ('Panaji', 'Raichur'): 457,
    ('Raichur', 'Panaji'): 457,
    ('Raichur', 'Tirupathi'): 453,
    ('Tirupathi', 'Raichur'): 453,
    ('Raichur', 'Kurnool'): 100,
    ('Kurnool', 'Raichur'): 100,
    ('Mangalore', 'Kozhikode'): 233,
    ('Kozhikode', 'Mangalore'): 233,
    ('Mangalore', 'Bangalore'): 352,
    ('Bangalore', 'Mangalore'): 352,
    ('Kozhikode', 'Bangalore'): 356,
    ('Bangalore', 'Kozhikode'): 356,
    ('Bangalore', 'Chennai'): 346,
    ('Chennai', 'Bangalore'): 346,
    ('Bangalore', 'Bellari'): 153,
    ('Bellari', 'Bangalore'): 153,
    ('Bellari', 'Tirupathi'): 379,
    ('Tirupathi', 'Bellari'): 379,
    ('Tirupathi', 'Chennai'): 153,
    ('Chennai', 'Tirupathi'): 153,
    ('Kurnool', 'Nellore'): 325,
    ('Nellore', 'Kurnool'): 325,
    ('Nellore', 'Chennai'): 175,
    ('Chennai', 'Nellore'): 175
}

graph = {
    "Panaji": {"Mangalore": 365 , "Raichur": 457},
    "Mangalore": {"Panaji": 365, "Bangalore": 352, "Kozhikode": 233},
    "Bangalore": {"Mangalore": 352, "Kozhikode": 356, "Bellari": 153, "Chennai": 346},
    "Kozhikode": {"Mangalore": 233, "Bangalore": 356},
    "Tirupathi": {"Bellari": 379, "Raichur": 453, "Chennai": 153},
    "Nellore": {"Kurnool": 325, "Chennai": 175},
    "Chennai": {"Tirupathi": 153, "Nellore": 175,"Bangalore": 346},
    "Raichur": {"Panaji": 457,"Tirupathi": 453,"Kurnool": 100},
    "Bellari": {"Bangalore": 153,"Tirupathi": 379},
    "Kurnool": {"Nellore": 325,"Raichur": 100}
}

# Function to generate a random valid path between start and end cities
def generate_random_valid_path(start, end, cities):
    path = [start]
    while len(path) < len(cities):
        possible_moves = [city for city in cities if city not in path]
        next_city = random.choice(possible_moves)
        path.append(next_city)
    path.append(end)
    return path

def generate_valid_path(start_city, goal_city, graph):
    """
    Generate a valid path from start to city to goal city based on map connections.
    
    :param start_city: The starting city for the path.
    :param goal_city: The goal city for the path.
    :param map_connections: A dictionary or similar structure representing valid connections between cities.
    :return: A list representing a valid path.
    """
    path = [start_city]
    current_city = start_city
    while current_city != goal_city:
        # Get valid next steps from current city
        next_cities = [city for city in graph[current_city] if city not in path]
        print("next citites ",next_cities)
        if not next_cities:  # No valid next city, break to avoid infinite loop
            break
        next_city = random.choice(next_cities)
        path.append(next_city)
        current_city = next_city
    return path


# Function to create the initial state for ga: a population of paths
def set_initial_state_for_ga(start, end, cities, population_size):
    # Initialize the population with random paths
    population = []
    for _ in range(population_size):
        path = generate_valid_path(start, end, cities)
        population.append(path)
    return population



def calculate_total_distance(path, distances):
    """
    Calculate the total distance of the given path based on the road distances.
    
    :param path: A list of tuples representing the path (e.g., [('City1', 'City2'), ('City2', 'City3'), ...]).
    :param distances: A dictionary with city tuples as keys and distances as values.
    :return: The total distance of the path.
    """
    total_distance = 0
    for i in range(len(path) - 1):
        city_pair = (path[i], path[i+1])
        total_distance += distances.get(city_pair, 0)
    return total_distance

def fitness_function(path, distances):
    total_distance = calculate_total_distance(path, distances)
    if total_distance <= 0:
        return 0  # Or a small epsilon value if you want to avoid zero fitness
    return 1 / total_distance

def is_path_valid(path, graph):
    for i in range(len(path) - 1):
        if path[i+1] not in graph[path[i]]:
            return False
    return True

def repair_path(path, start_city, goal_city, graph):
    repaired_path = [start_city]
    for current_city in path:
        if repaired_path[-1] == goal_city:
            break
        next_cities = graph.get(repaired_path[-1], [])
        if current_city in next_cities:
            repaired_path.append(current_city)
        else:
            # Find a valid next city that is not already in the repaired_path
            # This is a simplistic approach; more sophisticated logic may be needed
            for possible_next in next_cities:
                if possible_next not in repaired_path:
                    repaired_path.append(possible_next)
                    break
    if repaired_path[-1] != goal_city:
        repaired_path.append(goal_city)  # Ensure the path ends at the goal city
    return repaired_path

def selection(population, fitness_scores, selection_size):
    # Normalize fitness scores to use as probabilities for selection
    total_fitness = sum(fitness_scores)
    
    # Ensure total fitness is greater than zero and is finite
    if total_fitness <= 0 or not math.isfinite(total_fitness):
        raise ValueError("Invalid total fitness score: Must be greater than zero and finite.")
    
    selection_probabilities = [score / total_fitness for score in fitness_scores]
    
    # Select individuals based on their normalized fitness scores
    selected_indices = random.choices(
        range(len(population)), 
        weights=selection_probabilities, 
        k=selection_size
    )
    
    # Create a new list containing the selected individuals
    selected_population = [population[i] for i in selected_indices]
    return selected_population

def crossover(parent1, parent2, cities):
    # Ensure paths are long enough for crossover
    min_length = min(len(parent1), len(parent2))
    if min_length <= 3:
        # If paths are too short, return them as is or handle differently
        return parent1, parent2

    # Proceed with crossover
    crossover_point = random.randint(1, min_length - 2)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

    # Ensure the offspring are valid paths
    offspring1 = [city for city in cities if city in offspring1]
    offspring2 = [city for city in cities if city in offspring2]

    return offspring1, offspring2



def mutate(path, mutation_rate):
    """
    Mutate a path by swapping two cities with a given probability.
    
    :param path: The path to mutate.
    :param mutation_rate: The probability of mutating a given gene.
    :return: The mutated path.
    """
    mutated_path = path.copy()
    for i in range(len(mutated_path)):
        if random.random() < mutation_rate:
            swap_index = random.randint(0, len(mutated_path) - 1)
            # Swap the cities at i and swap_index
            mutated_path[i], mutated_path[swap_index] = mutated_path[swap_index], mutated_path[i]
    return mutated_path

def is_goal_reached_for_ga(path, goal_city):
    """
    Check if the goal city has been reached in the given path.
    
    :param path: A list representing the path taken, with each element being a city name.
    :param goal_city: The goal city name as a string.
    :return: True if the goal city is the last city in the path, False otherwise.
    """
    # Check if the last city in the path is the goal city
    return path[-1] == goal_city

import random

def genetic_algorithm_with_elitism(cities, distances, population_size, generations, crossover_rate, mutation_rate, goal_city, start_city,graph,number_of_elites=1):
    """
    Implements a Genetic Algorithm with elitism to find the most optimal path to the goal city.
    
    :param cities: List of all cities in the problem.
    :param distances: Dictionary with distances between each pair of cities.
    :param population_size: Number of paths in each generation.
    :param generations: Maximum number of generations to evolve.
    :param crossover_rate: Probability of applying crossover.
    :param mutation_rate: Probability of applying mutation.
    :param goal_city: The destination city for the pathfinding problem.
    :param number_of_elites: The number of top individuals to carry over directly to the next generation.
    :return: The most optimal path found and its distance.
    """
    # Initialize the population
    population = set_initial_state_for_ga(start_city, goal_city, graph, population_size)
    print("initial population ",population)
    best_path = None
    best_fitness = 0
    
    for generation in range(generations):
        # Evaluate the fitness of each individual in the population
        fitness_scores = [fitness_function(path, distances) for path in population]


    
        # Preserve the top individuals as elites
        elites_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:number_of_elites]
        elites = [population[i] for i in elites_indices]
        
        # Selection (excluding elites if they are not to be mutated)
        parents = selection(population, fitness_scores, len(population) // 2)
        
        # Adjusting the parent list to ensure an even number for crossover
        if len(parents) % 2 != 0:
            # Remove the last parent if the count is odd
            # You might choose a different strategy based on your specific requirements
            parents.pop()
        
        # Crossover and Mutation
        offspring = []
        # Example of applying the check and repair in the main GA loop
        for _ in range(0, len(parents), 2):
            if random.random() < crossover_rate:
                child1, child2 = crossover(parents[_], parents[_+1], cities)
            else:
                child1, child2 = parents[_], parents[_+1]

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            if not is_path_valid(child1, graph):
                child1 = repair_path(child1, start_city, goal_city, graph)
            if not is_path_valid(child2, graph):
                child2 = repair_path(child2, start_city, goal_city, graph)

            offspring.extend([child1, child2])
        
        # Combine elites with offspring to form the new generation
        new_population = elites + offspring[:population_size - len(elites)]
        population = new_population
        
        # Update best path if a new best is found
        for elite in elites:
            elite_fitness = fitness_function(elite, distances)
            if is_goal_reached_for_ga(elite, goal_city) and elite_fitness > best_fitness:
                best_path = elite
                best_fitness = elite_fitness
        
        print(f"Generation {generation + 1}: Best Path Distance = {1 / best_fitness if best_fitness else 'Not found'}")
    
    return best_path, 1 / best_fitness if best_fitness else None

def get_start_end_states(cities):
    """
    Prompt the user to select the start and end states for the journey by number instead of typing names.

    :param cities: A list of available cities.
    :return: A tuple containing the start and end states.
    """
    # Display cities with numbers
    for i, city in enumerate(cities, 1):
        print(f"{i}. {city}")

    while True:
        try:
            start_index = int(input("Select the number for the start city: ")) - 1
            if start_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            end_index = int(input("Select the number for the destination city: ")) - 1
            if end_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            if start_index == end_index:
                print("Start and destination cities cannot be the same. Please select different cities.")
                continue

            start = cities[start_index]
            end = cities[end_index]

            return start, end
        except ValueError:
            print("Invalid input. Please enter a number.")

# # Assuming a list of cities based on the given problem
# cities_list = ["Panaji", "Raichur", "Mangalore", "Bellari", "Tirupathi", "Kurnool", "Kozhikode", "Bangalore", "Nellore", "Chennai"]

start_city, goal_city = get_start_end_states(cities)

population_size = 200

# initial_population = set_initial_state_for_ga(start_city, goal_city, cities, population_size)
# initial_population
# # Define Genetic Algorithm parameters

generations = 100
crossover_rate = 0.8
mutation_rate = 0.2

# Invoke the Genetic Algorithm
best_path, best_distance = genetic_algorithm_with_elitism(
    cities, distances, population_size, generations, crossover_rate, mutation_rate, goal_city,
start_city,graph)

# Print the solution
if best_path and best_distance:
    print(f"Solution Found:\nPath: {' -> '.join(best_path)}\nCost (Total Distance): {best_distance} km")
else:
    print("No solution found within the given constraints.")

1. Panaji
2. Raichur
3. Tirupathi
4. Chennai
5. Mangalore
6. Kozhikode
7. Bangalore
8. Bellari
9. Kurnool
10. Nellore
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Nellore']
next citites  ['Chennai']
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Bellari', 'Chennai']
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Bellari', 'Chennai']
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Nellore']
next citites  ['Chennai']
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Bellari', 'Chennai']
next citites  ['Bangalore']
next citites  ['Mangalore', 'Kozhikode', 'Chennai']
next citites  ['Kozhikode']
next citites  []
next citites  ['Mangalore', 'Raichur']
next citites  ['Tirupathi', 'Kurnool']
next citites  ['Nellore']
next citites  ['Chennai']
next citites  ['Mangal